<a href="https://colab.research.google.com/github/MuftiqurBista1111/Unisnacks_Data_Forecasting/blob/main/UpdatingDate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from datetime import date
import datetime
from dateutil.relativedelta import relativedelta
from datetime import timedelta
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
import itertools
import math
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [ ]:
drive.mount('/content/drive')
data =  pd.read_csv("/content/drive/MyDrive/Unisnacks/Sales Order Custom Search Bista Results 637 - For Import Export demo.csv", parse_dates=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data2 = pd.read_csv("/content/drive/MyDrive/Unisnacks/updated_05.csv", parse_dates=True)

In [ ]:
data_new = data.loc[:,['Customer Name','Date','Quantity','Item : Internal ID','Document Number']]
data_new.rename(columns={'Item : Internal ID': 'Product_Id', 'Name':'Customer_Name'}, inplace=True)
result = data_new.groupby(['Customer Name', 'Product_Id', 'Date','Document Number']).size().reset_index()
result.head(6)

,Customer Name,Product_Id,Date,Document Number,0
0,1 Test Customer,2142.0,17/02/2023,SO142714,1
1,1 Test Customer,2143.0,17/02/2023,SO142714,1
2,1 Test Customer,2144.0,17/02/2023,SO142714,1
3,100 AJ world foods limited,490.0,14/01/2022,SO9745,1
4,100 AJ world foods limited,493.0,14/01/2022,SO9745,1
5,100 AJ world foods limited,507.0,14/01/2022,SO9745,1


In [ ]:
all_customer = set(data_new['Customer Name'])
all_product = set(result['Product_Id'])
all_document = set(result['Document Number'])
print(len(all_customer), len(all_product), len(all_document))

1316 1123 21559


In [ ]:
from datetime import datetime
def estimated_next_date(df):

    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    unique_dates = df['Date'].sort_values().unique()
    unique_dates_df = pd.DataFrame(unique_dates, columns=['Date'])
    if len(unique_dates_df) == 2:
        print("INSIDE HERE")
        last_date = unique_dates_df['Date'].iloc[-1]
        one_year_ago = pd.Timestamp(datetime.now() - pd.DateOffset(years=1))
        if last_date <= one_year_ago:
            print("NOT ENOUGH DATA TO ESTIMATE THE NEXT PREDICTION DATE")
            return None

    if len(unique_dates_df) == 1:
        print("NOT ENOUGH DATA TO ESTIMATE THE NEXT PREDICTION DATE")
        return None
    unique_dates_df['Difference'] = unique_dates_df['Date'].diff().dt.days
    average_difference = unique_dates_df['Difference'].mean()

    last_date = unique_dates_df['Date'].iloc[-1]
    estimated_next_date = last_date + pd.Timedelta(days=average_difference)

    current_date = pd.Timestamp(datetime.now()).date()
    while estimated_next_date.date() <= current_date:
        estimated_next_date += pd.Timedelta(days=average_difference)
    new_row = pd.DataFrame([[estimated_next_date, average_difference]], columns=['Date', 'Difference'])
    unique_dates_df = pd.concat([unique_dates_df, new_row], ignore_index=True)
    unique_dates_df['Date'] = unique_dates_df['Date'].dt.date
    return unique_dates_df['Date'].iloc[-1]

In [ ]:
customer = "DDX001 Matthews  Foods - Dundee"
filtered_rows = result[(result['Customer Name'] == customer)]

next_date = estimated_next_date(filtered_rows)
print(next_date)

2024-06-11


In [ ]:
filtered_rows.head(5)

,Customer Name,Product_Id,Date,Document Number,0
86191,DDX001 Matthews Foods - Dundee,-2.0,2022-02-14,SO10071,1
86192,DDX001 Matthews Foods - Dundee,-2.0,2022-08-22,SO12383,2
86193,DDX001 Matthews Foods - Dundee,409.0,2021-12-01,SO9318,1
86194,DDX001 Matthews Foods - Dundee,409.0,2022-12-03,SO13768,1
86195,DDX001 Matthews Foods - Dundee,409.0,2022-01-04,SO9602,1


In [ ]:
# Extract unique customers from data2
unique_customers_data2 = data2['Customer Name'].unique()


In [ ]:
# Define the estimated_next_date function
def estimated_next_date(df):
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    unique_dates = df['Date'].sort_values().unique()
    unique_dates_df = pd.DataFrame(unique_dates, columns=['Date'])

    if len(unique_dates_df) < 2:
        print("NOT ENOUGH DATA TO ESTIMATE THE NEXT PREDICTION DATE")
        return None

    unique_dates_df['Difference'] = unique_dates_df['Date'].diff().dt.days
    average_difference = unique_dates_df['Difference'].mean()

    last_date = unique_dates_df['Date'].iloc[-1]
    estimated_next_date = last_date + pd.Timedelta(days=average_difference)

    current_date = pd.Timestamp(datetime.now()).date()
    while estimated_next_date.date() <= current_date:
        estimated_next_date += pd.Timedelta(days=average_difference)

    new_row = pd.DataFrame([[estimated_next_date, average_difference]], columns=['Date', 'Difference'])
    unique_dates_df = pd.concat([unique_dates_df, new_row], ignore_index=True)
    unique_dates_df['Date'] = unique_dates_df['Date'].dt.date
    return unique_dates_df['Date'].iloc[-1]

# Extract unique customers from data2
unique_customers_data2 = data2['Customer Name'].unique()

# Predict next date for each unique customer and print
predicted_dates = []
for customer in unique_customers_data2:
    filtered_rows = result[(result['Customer Name'] == customer)]
    next_date = estimated_next_date(filtered_rows)
    if next_date:
        predicted_dates.append({'Customer Name': customer, 'Predicted Date': next_date})
        print(f"Customer: {customer}, Predicted Next Date: {next_date}")

# Create a dataframe with predicted dates
predicted_dates_df = pd.DataFrame(predicted_dates)

# Update the Date field in data2 dataframe
data2_updated = data2.copy()
for index, row in predicted_dates_df.iterrows():
    data2_updated.loc[data2_updated['Customer Name'] == row['Customer Name'], 'Date'] = row['Predicted Date']

# Save the final output as a CSV file
data2_updated.to_csv("/content/drive/MyDrive/Unisnacks/updated_with_predictions.csv", index=False)

Customer: BXX012 New Harvest Wholesale Ltd, Predicted Next Date: 2024-06-04
Customer: ALX005 VRD Retail Limited, Predicted Next Date: 2024-06-10
Customer: TRX002 Chan's Oriental Food & Products, Predicted Next Date: 2024-07-04
Customer: SEX020 M&D Japanese Food Takeaway, Predicted Next Date: 2024-07-01
Customer: SMX003 Sitha Ram, Predicted Next Date: 2024-06-24
Customer: LEX009 Goldens (Leicester) Ltd T/A Yours Supermarket, Predicted Next Date: 2024-06-25
Customer: CRX002 Hoo Hing - Mitcham, Predicted Next Date: 2024-07-30
Customer: EHX017 V TEVIOT LIMITED T/A Ting Thai Caravan, Predicted Next Date: 2024-06-05
Customer: BSX008 Holleys Fine Foods Ltd, Predicted Next Date: 2024-05-31
Customer: GUX006 Dhampus Ltd., Predicted Next Date: 2024-06-24
Customer: MKX010 Lamberts Bedford Ltd, Predicted Next Date: 2024-08-16
Customer: NEX007 Tuk Tuk Market, Predicted Next Date: 2024-06-24
Customer: NWX012 Jin Yu Lin Ltd T/A Eat Tokyo, Predicted Next Date: 2024-06-06
Customer: SEX019 Ly Trang Orien